In [1]:
from _shared import *
from tqdm.notebook import tqdm

## Задача 1. Количество параллельных экспериментов

Сколько несовместных независимых экспериментов можно запустить одновременно, если за время эксперимента собираем 10000 наблюдений?

Если решаем запустить 10 экспериментов, то на каждый эксперимент можно выделить по 1000 наблюдений, размер групп будет равен 500.

Параметры экспериментов:
- проверяем гипотезу о равенстве средних;
- уровень значимости — 0.05;
- допустимая вероятность ошибки II рода — 0.1;
- ожидаемый эффект — увеличение значений на 3%;
- способ добавления эффекта в синтетических А/Б экспериментах — умножение на константу.

Будем считать, что распределение измеряемых величин является нормальным распределением со средним 100 и стандартным отклонением 10.

В качестве ответа введите максимально возможное количество экспериментов, которое можно запустить с указанными выше параметрами.

In [2]:
obs_num = 10_000
test_ = stats.ttest_ind

alpha_ = 0.05
beta_ = 0.1
effect_ = +0.03

mean_ = 100.0
std_ = 10.0

In [3]:
# df = pd.DataFrame(np.random.normal(loc=mean_, scale=std_, size=obs_num), columns=['metric_a']) #Let index be the user_id.
# df['metric_b'] = df['metric_a'] * (1 + effect_)

# Number of test equals half the number of observation divided by required sample size (since we need 2 groups).
res_1 = int(np.floor(obs_num / get_sample_size_rel(mu=mean_, std=std_, eff=effect_, alpha=alpha_, beta=beta_)) / 2)
res_1

21

In [4]:
def check_experiment_design_ind(number_of_experiments, N=10_000):
    group_size = int((obs_num / number_of_experiments) / 2)
    aa_res, ab_res = [], []

    for _ in tqdm(range(N)):
        metric_a, metric_b = np.random.normal(loc=mean_, scale=std_, size=(2, group_size))
        metric_b_eff = metric_b * (1.0 + effect_)

        aa_res.append(test_(metric_a, metric_b).pvalue)
        ab_res.append(test_(metric_a, metric_b_eff).pvalue)

    type_i_error_rate = (np.array(aa_res) < alpha_).mean()
    type_ii_error_rate = (np.array(ab_res) >= alpha_).mean()
    
    return (type_i_error_rate, type_ii_error_rate)


res_string = '{:.0f}: I order: {:.4f} [{}]; II order: {:.4f} [{}]'

In [5]:
for n in (res_1-1, res_1, res_1+1):
    p_i, p_ii = check_experiment_design_ind(n, N=10_000)
    print(res_string.format(n, p_i, p_i < alpha_, p_ii, p_ii < beta_))

  0%|          | 0/10000 [00:00<?, ?it/s]

20: I order: 0.0491 [True]; II order: 0.0925 [True]


  0%|          | 0/10000 [00:00<?, ?it/s]

21: I order: 0.0510 [False]; II order: 0.1052 [False]


  0%|          | 0/10000 [00:00<?, ?it/s]

22: I order: 0.0495 [True]; II order: 0.1209 [False]


In [6]:
# Solution.
import numpy as np
from scipy import stats
from tqdm.notebook import tqdm

# параметры эксперимента
total_size = 10000
mean_ = 100
std_ = 10
effect = 0.03
alpha = 0.05
beta = 0.1


def estimate_sample_size(effect, std, alpha, beta):
    """Оценка необходимого размер групп."""
    t_alpha = stats.norm.ppf(1 - alpha / 2, loc=0, scale=1)
    t_beta = stats.norm.ppf(1 - beta, loc=0, scale=1)
    var = 2 * std ** 2
    sample_size = int((t_alpha + t_beta) ** 2 * var / (effect ** 2))
    return sample_size


# оценим необходимый размер групп
sample_size = estimate_sample_size(effect * 100, 10, alpha, beta)
print(f'sample_size = {sample_size}')
# вычислим количество экспериментов
count_exp = total_size / (sample_size * 2)
print(f'count_exp = {count_exp:0.1f}')


def estimate_ci_bernoulli(p, n, alpha=0.05):
    """Доверительный интервал для Бернуллиевской случайной величины."""
    t = stats.norm.ppf(1 - alpha / 2, loc=0, scale=1)
    std_n = np.sqrt(p * (1 - p) / n)
    return p - t * std_n, p + t * std_n

# Проверим, что при 21 эксперимента ошибки контролируются на заданных уровнях, а при 22 экспериментах нет.

for count_exp in [21, 22]:
    errors_aa = []
    errors_ab = []
    sample_size = int(total_size / (int(count_exp) * 2))
    for _ in tqdm(range(10000)):
        a, b = np.random.normal(mean_, std_, (2, sample_size,))
        b_effect = b * (1 + effect)
        errors_aa.append(stats.ttest_ind(a, b).pvalue < alpha)
        errors_ab.append(stats.ttest_ind(a, b_effect).pvalue >= alpha)

    estimated_first_type_error = np.mean(errors_aa)
    estimated_second_type_error = np.mean(errors_ab)
    ci_first = estimate_ci_bernoulli(estimated_first_type_error, len(errors_aa))
    ci_second = estimate_ci_bernoulli(estimated_second_type_error, len(errors_ab))
    print(f'count_exp = {count_exp}')
    print(f'sample_size = {sample_size}')
    print(f'оценка вероятности ошибки I рода = {estimated_first_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_first[0]:0.4f}, {ci_first[1]:0.4f}]')
    print(f'оценка вероятности ошибки II рода = {estimated_second_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_second[0]:0.4f}, {ci_second[1]:0.4f}]')


sample_size = 233
count_exp = 21.5


  0%|          | 0/10000 [00:00<?, ?it/s]

count_exp = 21
sample_size = 238
оценка вероятности ошибки I рода = 0.0494
  доверительный интервал = [0.0452, 0.0536]
оценка вероятности ошибки II рода = 0.1070
  доверительный интервал = [0.1009, 0.1131]


  0%|          | 0/10000 [00:00<?, ?it/s]

count_exp = 22
sample_size = 227
оценка вероятности ошибки I рода = 0.0528
  доверительный интервал = [0.0484, 0.0572]
оценка вероятности ошибки II рода = 0.1189
  доверительный интервал = [0.1126, 0.1252]


## Задача 2. Количество параллельных экспериментов — 2

Задача похожа на предыдущую, только теперь __решение принимается не независимо для каждого эксперимента.__
Например, у нас есть 5 текстов для маркетинговой рассылки, хотим проверить, какой эффективнее работает и работает ли вообще.

__Алгоритм будет следующий:__
1. Формируем непересекающиеся контрольные и экспериментальные группы для каждого из 5 вариантов.
2. Проводим параллельно 5 экспериментов.
3. С помощью метода Холма определяем, в каких экспериментах были статистически значимые отличия.
4. Если значимых отличий не обнаружено, то говорим, что эффекта нет, все варианты отклоняем.
5. Если значимые отличия обнаружены, то из вариантов со значимым эффектом выбираем вариант с наименьшим значением p-value, будем использовать его.

Будем считать, что __совершена ошибка I рода__, если найдены значимые отличия, когда на самом деле их не было ни в одном из вариантов.

Будем считать, что __совершена ошибка II рода__, если:
- либо не найдено значимых отличий, когда на самом деле они были;
- либо выбранный для дальнейшего использования вариант на самом деле был без эффекта, при этом были варианты с эффектом.

__Параметры экспериментов:__
- проверяем гипотезу о равенстве средних;
- уровень значимости — 0.05;
- допустимая вероятность ошибки II рода — 0.1;
- ожидаемый эффект — увеличение значений на 3%;
- способ добавления эффекта в синтетических А/Б экспериментах — умножение на константу.

Замечание: при оценке вероятности ошибки II рода нужно рассматривать худший сценарий, когда эффект есть только в одном из экспериментов. Чем в большем количестве экспериментов будет эффект, тем меньше будет вероятность ошибки II рода.

Будем считать, что распределение измеряемых величин является нормальным распределением со средним 100 и стандартным отклонением 10.

В качестве ответа введите максимально возможное количество экспериментов, которое можно запустить с указанными выше параметрами.



In [7]:
def bonferroni_method(pvalues, alpha=0.05):
    """
    Carries out Bonferroni correction.

    pvalues - list of pvalues.
    alpha -  level of significance.
    return - array of 0/1 flags indicating presence/absence of effect.
    """
    pvalues = np.array(pvalues)
    
    return np.array(pvalues < (alpha / pvalues.shape[0])).astype(int)


def holm_method(pvalues, alpha=0.05):
    """
    Carries out Holm method correction.

    pvalues - list of pvalues.
    alpha -  level of significance.
    return - array of 0/1 flags indicating presence/absence of effect.
    """
    n = len(pvalues)
    
    alphas = alpha / np.arange(n, 0, -1) #Array of alpha divided by indices.
    sorted_indices = np.argsort(pvalues)
    res = np.zeros(n)
    
    for (i, pv_i) in enumerate(sorted_indices):
        if pvalues[pv_i] < alphas[i]:
            res[pv_i] = 1
        else:
            break

    return res.astype(int)

In [8]:
def check_experiment_design_dep(number_of_experiments, N=10_000):
    group_size = int(obs_num / 2)
    experiment_size = int(group_size / number_of_experiments)
    aa_res, ab_res = [], []
    
    for _ in tqdm(range(N)):
        metric_a, metric_b = np.random.normal(loc=mean_, scale=std_, size=(2, group_size))
        
        metric_b_eff = metric_b.copy()
        metric_b_eff[:experiment_size] *= (1.0 + effect_)
        
        aa_step, ab_step = [], []
        
        for i in range(number_of_experiments):
            beg_ = i * experiment_size
            end_ = beg_ + experiment_size - 1
            
            aa_step.append(test_(metric_a[beg_:end_], metric_b[beg_:end_]).pvalue)
            ab_step.append(test_(metric_a[beg_:end_], metric_b_eff[beg_:end_]).pvalue)
        
        #If any of AA is 1, then type I error: found effect where none exists.
        aa_res.append(correction_(aa_step).max() == 1)
        
        #If the first one (the only one with effect) is not 1, then type II error: did not find effect where it exists.
        #If the first one does not have the minimal pvalue, then modified type II error: chose not the best variant.
        ab_res.append(correction_(ab_step)[0] != 1 or ab_step[0] != np.min(ab_step))
        
    type_i_error_rate = np.mean(aa_res)
    type_ii_error_rate = np.mean(ab_res)
    
    return (type_i_error_rate, type_ii_error_rate)


correction_ = holm_method

In [9]:
for n in (res_1-1, res_1, res_1+1):
    p_i, p_ii = check_experiment_design_dep(n, N=1_000)
    print(res_string.format(n, p_i, p_i < alpha_, p_ii, p_ii < beta_))

  0%|          | 0/1000 [00:00<?, ?it/s]

20: I order: 0.0570 [False]; II order: 0.4030 [False]


  0%|          | 0/1000 [00:00<?, ?it/s]

21: I order: 0.0560 [False]; II order: 0.4530 [False]


  0%|          | 0/1000 [00:00<?, ?it/s]

22: I order: 0.0470 [True]; II order: 0.4830 [False]


In [10]:
# for n in range(1, res_1+1): #Used for initial run, gave 12.
for n in range(5, 14+1):
    p_i, p_ii = check_experiment_design_dep(n, N=10_000)
    print(res_string.format(n, p_i, p_i < alpha_, p_ii, p_ii < beta_))

  0%|          | 0/10000 [00:00<?, ?it/s]

5: I order: 0.0493 [True]; II order: 0.0000 [True]


  0%|          | 0/10000 [00:00<?, ?it/s]

6: I order: 0.0510 [False]; II order: 0.0007 [True]


  0%|          | 0/10000 [00:00<?, ?it/s]

...

12: I order: 0.0443 [True]; II order: 0.0865 [True]


  0%|          | 0/10000 [00:00<?, ?it/s]

13: I order: 0.0478 [True]; II order: 0.1221 [False]


  0%|          | 0/10000 [00:00<?, ?it/s]

14: I order: 0.0503 [False]; II order: 0.1623 [False]


In [11]:
"""12: I order: 0.0490 [True]; II order: 0.0960 [True]"""
#Chose the maximum one with acceptable II order rate. Lucky it was the right answer from the first try.
res_2 = 12

In [12]:
# Solution
def method_holm(pvalues, alpha=0.05):
    """Применяет метод Холма для проверки значимости изменений.
    
    pvalues - List[float] - список pvalue.
    alpha - float, уровень значимости.
    return - np.array, массив из нулей и единиц, 0 - эффекта нет, 1 - эффект есть.
    """
    m = len(pvalues)
    array_alpha = np.arange(m, 0, -1)
    array_alpha = alpha / array_alpha
    sorted_pvalue_indexes = np.argsort(pvalues)
    res = np.zeros(m)
    for idx, pvalue_index in enumerate(sorted_pvalue_indexes):
        pvalue = pvalues[pvalue_index]
        alpha_ = array_alpha[idx]
        if pvalue < alpha_:
            res[pvalue_index] = 1
        else:
            break
    res = res.astype(int)
    return res

# Проверим, что при 12 эксперимента ошибки контролируются на заданных уровнях, а при 13 экспериментах нет.

for count_exp in [12, 13]:
    errors_aa = []
    errors_ab = []
    sample_size = int(total_size / (int(count_exp) * 2))
    for _ in tqdm(range(10000)):
        list_ab_values = [
            np.random.normal(mean_, std_, (2, sample_size))
            for _ in range(count_exp)
        ]
        # синтетический А/А тест
        pvalues = [stats.ttest_ind(a, b).pvalue for a, b in list_ab_values]
        aa_with_effect = method_holm(pvalues, alpha)
        errors_aa.append(np.sum(aa_with_effect) > 0)

        # Синтетический А/Б тест.
        # Достаточно проверить случай, когда эффект есть лишь в одном из экспериментов,
        # так как при наличии эффектов в большем кол-ве экспериментов ошибок II рода станет меньше.
        # Добавим эффект в первый эксперимент (не важно в какой добавлять, так как данные случайные)
        list_ab_values[0][1] *= 1 + effect
        pvalues = [stats.ttest_ind(a, b).pvalue for a, b in list_ab_values]
        ab_with_effect = method_holm(pvalues, alpha)
        if np.sum(ab_with_effect) == 0:
            # если эффектов не найдено, то это ошибка
            errors_ab.append(True)
        else:
            # если эффектов найден где его нет, то это ошибка
            errors_ab.append(np.min(pvalues) != pvalues[0])

    estimated_first_type_error = np.mean(errors_aa)
    estimated_second_type_error = np.mean(errors_ab)
    ci_first = estimate_ci_bernoulli(estimated_first_type_error, len(errors_aa))
    ci_second = estimate_ci_bernoulli(estimated_second_type_error, len(errors_ab))
    print(f'count_exp = {count_exp}')
    print(f'sample_size = {sample_size}')
    print(f'оценка вероятности ошибки I рода = {estimated_first_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_first[0]:0.4f}, {ci_first[1]:0.4f}]')
    print(f'оценка вероятности ошибки II рода = {estimated_second_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_second[0]:0.4f}, {ci_second[1]:0.4f}]')

  0%|          | 0/10000 [00:00<?, ?it/s]

count_exp = 12
sample_size = 416
оценка вероятности ошибки I рода = 0.0459
  доверительный интервал = [0.0418, 0.0500]
оценка вероятности ошибки II рода = 0.0859
  доверительный интервал = [0.0804, 0.0914]


  0%|          | 0/10000 [00:00<?, ?it/s]

count_exp = 13
sample_size = 384
оценка вероятности ошибки I рода = 0.0472
  доверительный интервал = [0.0430, 0.0514]
оценка вероятности ошибки II рода = 0.1208
  доверительный интервал = [0.1144, 0.1272]


Получили, что при 12 экспериментах всё корректно, а при 13 экспериментах вероятность ошибки II рода больше 0.1.

__Получается, если мы проводим множественное тестирование, то за раз можем проверить меньшее количество гипотез, чем если бы проверяли независимые гипотезы.__ Это логично, так как мы предъявляем более строгое требование к вероятностям ошибок первого рода при множественном тестировании. Из-за этого наш критерий дополнительно перестраховывается и реже говорит, что эффект есть, что приводит к увеличению вероятности ошибок II рода. Компенсировать увеличение вероятности ошибок II рода приходится за счёт увеличения размера групп.

